In [119]:
from lxml import html
import requests

In [120]:
page = requests.get('https://www.reddit.com/r/AdviceAnimals/comments/5y54am/my_uncle_is_an_awesome_boss/')
tree = html.fromstring(page.content)

In [121]:
comments = tree.xpath('//div[@class="md"]/p/text()')
#This will create a list of points 
points = tree.xpath('//span[@class="score unvoted"]/@title')

In [122]:
import pandas as pd
int_points_array = [int(numeric_string) for numeric_string in points]
a={'Comments':pd.Series(comments),'Points':pd.Series(int_points_array)}
df=pd.DataFrame(a)

In [123]:
df_sorted=df.sort('Points',ascending=False)[0:10]
df_sorted

c:\users\dell\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Comments,Points
17,This is a good fucking way to look at any poli...,7002.0
19,The customer isn't always right. I feel like t...,3982.0
18,"Politicical views dont matter, country of orig...",3581.0
0,"People like your uncle, the world needs more o...",2051.0
60,I would happily be your uncle's horse!,1720.0
1,If we forget about the politics of the situati...,1706.0
20,The saying,1365.0
59,Your uncle is legend. Thank you and your family,931.0
2,Let's not forget the politics of the situation...,923.0
96,You log out?,898.0


In [124]:
comments_array=df_sorted['Comments'].values[0:10]
comments_array

array([ 'This is a good fucking way to look at any political situation these days, strip it down to its basic form, and analys it there...',
       "The customer isn't always right. I feel like the only people who say that are past 2005 are customers or shitty managers.",
       'Politicical views dont matter, country of origin does not matter, throw all of that shit away, and think of the basic scenario as u stated "employe threatened by a custommer"',
       'People like your uncle, the world needs more of them. ',
       "I would happily be your uncle's horse!",
       "If we forget about the politics of the situation, it's a customer threatening an employee. That's something that should not be tolerated. ",
       'The saying ', 'Your uncle is legend. Thank you and your family',
       "Let's not forget the politics of the situation. I'm an immigrant just like your uncle's employee, and I appreciate employers that stand up against hate speech. ",
       'You log out?'], dtype=objec

In [125]:
from sklearn.feature_extraction.text import CountVectorizer
vect =CountVectorizer()

In [126]:
bag_words=vect.fit_transform(comments_array)

# Using Bag of words approach

In [127]:
import pandas as pd
pd.DataFrame(bag_words.toarray(),columns=vect.get_feature_names())
#From this no conclusion can be drawn as words like (is,a,the will be repeated many times)

,2005,about,against,all,always,an,analys,and,any,appreciate,...,uncle,up,views,way,we,who,world,would,you,your
0,0,0,0,0,0,0,1,1,1,0,...,0,0,0,1,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,1
5,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,2
8,0,0,1,0,0,1,0,1,0,1,...,1,1,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


# Using TfIDF approach

In [128]:
import math
from textblob import TextBlob as tb

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [129]:
document1=tb(comments_array[0])
document2=tb(comments_array[1])
document3=tb(comments_array[2])
document4=tb(comments_array[3])
document5=tb(comments_array[4])
document6=tb(comments_array[5])
document7=tb(comments_array[6])
document8=tb(comments_array[7])
document9=tb(comments_array[8])
document10=tb(comments_array[9])
bloblist = [document1, document2, document3,document4,document5,document6,document7,document8,document9,document10]
arr1=[]
for i, blob in enumerate(bloblist):
    #print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:10]:
        #print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))
        arr1.append(word)

# Tokenize the words ,removing stop words from it ,finding nouns in it.

In [130]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import word_tokenize,Text,pos_tag
arr2=[]
def sentence_verification(sentence):
    word_find=sentence
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(word_find)
    filtered_sentence=[]
    for w in word_tokens:
        if w not in stop_words:
                     filtered_sentence.append(w)
    #print (filtered_sentence)
    text = Text(filtered_sentence)
    tags = pos_tag(text)
    #print (tags)
    nouns = "NN NNP NNS".split()
    verbs = "VB VBD VBP VBG".split()
    for i in range(len(tags) -1):
        if tags[i][1] in nouns:
            #print (tags[i][0],tags[i][1])
            arr2.append(tags[i][0])

In [131]:
for i in range(len(comments_array)):
    sentence_verification(comments_array[i])
    #print(".....................................................................................................")

# Now we will take only those words which are in all the three approaches and that will be our final set of words.

In [132]:
list(set(arr1) & set(arr2))

['Thank',
 'speech',
 'matter',
 'form',
 'world',
 'People',
 'something',
 'uncle',
 'horse',
 'needs',
 'employers',
 'customers',
 'country',
 'threatening',
 'legend',
 'days',
 'people']